In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from Crypto.Hash import SHA256
import random


In [3]:

IMG_SIZE = (128, 128)  
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "Palm_train_dataset",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

test_data = test_datagen.flow_from_directory(
    "Palm_test_dataset",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)


class_indices = train_data.class_indices
class_labels = {v: k for k, v in class_indices.items()}  # Reverse dictionary


Found 1433 images belonging to 41 classes.
Found 470 images belonging to 41 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(class_indices), activation='softmax')  # Output classes = No. of folders
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 41)             │         5,289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,309,929 (12.63 MB)

 Trainable params: 3,309,929 (12.63 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
EPOCHS = 10
model.fit(train_data, validation_data=test_data, epochs=EPOCHS)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 38s 840ms/step - accuracy: 0.0383 - loss: 3.7246 - val_accuracy: 0.1872 - val_loss: 3.4154
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 36s 809ms/step - accuracy: 0.1422 - loss: 3.2463 - val_accuracy: 0.5596 - val_loss: 1.8964
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 36s 808ms/step - accuracy: 0.4598 - loss: 2.0710 - val_accuracy: 0.8234 - val_loss: 0.9299
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 36s 811ms/step - accuracy: 0.6095 - loss: 1.3315 - val_accuracy: 0.8979 - val_loss: 0.5190
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 38s 841ms/step - accuracy: 0.7528 - loss: 0.8636 - val_accuracy: 0.9426 - val_loss: 0.3060
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 38s 845ms/step - accuracy: 0.7960 - loss: 0.6718 - val_accuracy: 0.9426 - val_loss: 0.2082
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 40s 879ms/step - accuracy: 0.8402 - loss: 0.5783 - val_accuracy: 0.9660 - val_loss: 0.1272
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 874ms/step - accuracy: 0.8440 - loss: 0.4588 - val_accu

In [6]:
def extract_features(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    feature_vector = model.predict(img_array)[0]  # Extracted features
    predicted_class = np.argmax(feature_vector)  # Get class ID
    folder_name = class_labels[predicted_class]  # Get folder name

    return feature_vector, folder_name


In [7]:
def hash_commitment(commitment):
    return SHA256.new(commitment.tobytes()).hexdigest()

def zero_knowledge_proof(user_image_path):

    feature_vector, folder_name = extract_features(user_image_path)


    commitment_hash = hash_commitment(feature_vector)


    challenge = random.randint(0, 1)


    if challenge == 1:
        response = feature_vector  
    else:
        response = commitment_hash  


    if challenge == 1:
        verification_hash = hash_commitment(response)
        if verification_hash == commitment_hash:
            print(f"✅ Authentication Successful for {folder_name} (ZKP Verified)")
            print(f"🔹 Hash Value: {commitment_hash}")
        else:
            print("❌ Authentication Failed (ZKP Mismatch)")
    else:
        print(f"✅ Commitment Verified for {folder_name} (ZKP Step Passed)")
        print(f"🔹 Hash Value: {commitment_hash}")


In [15]:
zero_knowledge_proof("sample_user4.JPG")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
✅ Commitment Verified for 004 (ZKP Step Passed)
🔹 Hash Value: 0795279f3d7b0f8eeb7cac2351759d531219d3fe90879d22b7e562c977e5d979
